<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-24 10:41:10
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.40 C
-------------------
Today PnL: -34.64 K (-0.25%)
Current PnL: -19.23 L (-12.94%)
CY Booked + Current PnL: -7.79 L (-5.24%)
-------------------
Total profit:  1.40 L
Total loss:  -20.63 L
-------------------
Total Booked + Current PnL: 19.19 L (15.77%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 85.10 L (60.63%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,12.54,56.0,H-MC,3.77,194180.0,29510.0,13826.0,-1.13,17.92,7.12,26.31,89.0,2.13,1.38,27.33,X40N,NTT,AC
77,TTKPRESTIG,770.00,96.16,39.0,M-SC,2.92,84502.0,-16275.0,16368.0,-0.66,-16.15,19.37,0.09,245.0,-0.99,0.60,11.02,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,61.0,H-MC,9.91,110897.0,10393.0,18719.0,0.19,10.34,16.88,28.96,79.0,0.56,0.79,47.82,MH,ATH,METALS
57,RELIANCE,1533.00,-13.66,47.0,H-LC,4.45,216247.0,5881.0,22901.0,-0.26,2.80,10.59,13.68,37.0,0.26,1.54,19.78,XY25,NTT,REFINERIES
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-14.58,73.0,H-SC,15.47,229469.0,-41292.0,371235.0,1.83,-15.25,161.78,121.86,133.0,-0.11,1.64,79.51,AR,ATH,IT
69,SURYODAY,240.00,51.32,66.0,H-SC,1.06,138720.0,-40351.0,106079.0,1.83,-22.53,76.47,36.71,135.0,-0.38,0.99,37.64,XR,NTT,BANKS
0,5PAISA,593.00,124.76,48.0,H-SC,7.25,152402.0,-66094.0,94291.0,1.41,-30.25,61.87,12.90,153.0,-0.70,1.09,16.01,OX40N,NTT,FINANCE
68,STARHEALTH,761.00,11.22,60.0,H-SC,4.83,247928.0,-33195.0,166062.0,1.09,-11.81,66.98,47.26,144.0,-0.20,1.77,32.83,XY24,NTT,INSURANCE
62,SATIN,274.00,-27.73,51.0,H-SC,4.95,222592.0,-53692.0,196905.0,1.09,-19.43,88.46,51.83,126.0,-0.27,1.59,8.44,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4850.92,-54.64,27.0,H-SC,11.00,126723.0,3387.0,130373.0,-2.38,2.75,102.88,108.45,119.0,0.03,0.90,25.71,AR,ATH,MISC
7,ATULAUTO,844.00,3764.86,56.0,M-SC,5.98,120454.0,-26914.0,78729.0,-1.96,-18.26,65.36,35.16,236.0,-0.34,0.86,23.03,XY24,NTT,AUTO
16,CAMPUS,393.00,-25.94,48.0,M-SC,0.40,154370.0,-13405.0,69250.0,-1.84,-7.99,44.86,33.28,210.0,-0.19,1.10,19.65,XY24,NTT,FOOTWEAR
18,CERA,9475.00,-23.39,31.0,H-SC,2.17,139656.0,-36247.0,78263.0,-1.75,-20.61,56.04,23.89,149.0,-0.46,1.00,20.61,OX40N,NTT,CERAMICS
51,NESTLEIND,1377.00,-11.15,44.0,H-LC,4.45,273406.0,7980.0,48803.0,-1.67,3.01,17.85,21.40,11.0,0.16,1.95,10.31,XY25,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1239.71,76.0,M-SC,9.96,182245.0,1102.0,44358.0,-1.44,0.61,24.34,25.10,235.0,0.02,1.30,46.05,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.00,-21.57,42.0,H-MC,2.02,135001.0,-787.0,43538.0,-0.58,-0.58,32.25,31.48,107.0,-0.02,0.96,11.47,X40,ATH,INSURANCE
23,DABUR,735.00,-8.16,37.0,H-MC,6.79,248684.0,616.0,107780.0,-0.44,0.25,43.34,43.70,102.0,0.01,1.77,14.73,XY24,BTT,FMCG
52,PGHH,17973.08,-30.15,54.0,H-MC,3.67,202200.0,1380.0,67393.0,0.54,0.69,33.33,34.25,80.0,0.02,1.44,5.92,X40,ATH,FMCG
86,WIPRO,420.00,-13.34,42.0,M-LC,5.37,152204.0,1259.0,108202.0,-1.66,0.83,71.09,72.51,53.0,0.01,1.08,6.91,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.62,121716.0,-20763.0,40288.0,-0.29,-14.57,33.10,13.70,163.0,-0.52,0.87,43.66,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,96.16,39.0,M-SC,2.92,84502.0,-16275.0,16368.0,-0.66,-16.15,19.37,0.09,245.0,-0.99,0.60,11.02,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,52.0,H-SC,1.12,221913.0,-47754.0,84083.0,0.15,-17.71,37.89,13.47,138.0,-0.57,1.58,12.24,XY24,NTT,PAINTS
66,SIS,528.0,2104.66,43.0,H-SC,4.70,88970.0,-22062.0,45668.0,1.07,-19.87,51.33,21.26,156.0,-0.48,0.63,20.21,OX40N,NTT,MISC
18,CERA,9475.0,-23.39,31.0,H-SC,2.17,139656.0,-36247.0,78263.0,-1.75,-20.61,56.04,23.89,149.0,-0.46,1.00,20.61,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1239.71,76.0,M-SC,9.96,182245.0,1102.0,44358.0,-1.44,0.61,24.34,25.10,235.0,0.02,1.30,46.05,XY24,NTT,HEALTHCARE
60,SAIL,228.00,49.96,72.0,M-MC,9.79,237309.0,12347.0,155319.0,0.37,5.49,65.45,74.53,192.0,0.08,1.69,39.51,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.54,56.0,H-MC,3.77,194180.0,29510.0,13826.0,-1.13,17.92,7.12,26.31,89.0,2.13,1.38,27.33,X40N,NTT,AC
84,VOLTAS,1530.00,-2.34,44.0,H-MC,2.96,204825.0,13083.0,24681.0,-0.52,6.82,12.05,19.69,99.0,0.53,1.46,14.08,XY25,NTT,AC
17,CAMS,5211.76,-3.39,51.0,H-SC,2.26,109931.0,7927.0,36002.0,-1.51,7.77,32.75,43.06,122.0,0.22,0.78,26.22,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,61.0,H-MC,9.91,110897.0,10393.0,18719.0,0.19,10.34,16.88,28.96,79.0,0.56,0.79,47.82,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.55,46.0,M-SC,1.49,98880.0,7382.0,71372.0,-0.19,8.07,72.18,86.07,223.0,0.10,0.70,43.59,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,69.0,H-MC,7.02,186759.0,6951.0,105463.0,0.75,3.87,56.47,62.52,88.0,0.07,1.33,39.27,XR,NTT,BANKS
37,INDIAMART,4850.92,-54.64,27.0,H-SC,11.00,126723.0,3387.0,130373.0,-2.38,2.75,102.88,108.45,119.0,0.03,0.90,25.71,AR,ATH,MISC
86,WIPRO,420.00,-13.34,42.0,M-LC,5.37,152204.0,1259.0,108202.0,-1.66,0.83,71.09,72.51,53.0,0.01,1.08,6.91,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-19.29,34.0,H-LC,7.69,290896.0,-24946.0,137623.0,-0.56,-7.90,47.31,35.67,5.0,-0.18,2.07,4.86,X40N,ATH,FMCG
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
74,TCS,4476.75,-28.04,41.0,H-LC,12.12,286362.0,-59594.0,134447.0,-0.52,-17.23,46.95,21.64,1.0,-0.44,2.04,1.45,X40,ATH,IT
30,HCLTECH,1943.91,-1.08,42.0,H-LC,9.92,225609.0,-16315.0,79595.0,0.50,-6.74,35.28,26.15,8.0,-0.20,1.61,6.61,X40,ATH,IT
31,HINDUNILVR,2922.00,-9.52,42.0,H-LC,9.95,253880.0,-3183.0,38310.0,0.66,-1.24,15.09,13.67,24.0,-0.08,1.81,18.92,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.20,47.0,H-LC,1.55,235215.0,-10936.0,90134.0,-0.67,-4.44,38.32,32.18,16.0,-0.12,1.68,30.00,X200,ATH,IT
42,ITC,452.0,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
65,SIEMENS,4671.5,0.17,52.0,H-LC,3.07,160900.0,-25195.0,72679.0,-0.69,-13.54,45.17,25.51,15.0,-0.35,1.15,18.57,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-11.15,44.0,H-LC,4.45,273406.0,7980.0,48803.0,-1.67,3.01,17.85,21.40,11.0,0.16,1.95,10.31,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,47.0,H-LC,4.45,216247.0,5881.0,22901.0,-0.26,2.80,10.59,13.68,37.0,0.26,1.54,19.78,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.04,41.0,H-LC,12.12,286362.0,-59594.0,134447.0,-0.52,-17.23,46.95,21.64,1.0,-0.44,2.04,1.45,X40,ATH,IT
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
83,VBL,671.64,-19.29,34.0,H-LC,7.69,290896.0,-24946.0,137623.0,-0.56,-7.90,47.31,35.67,5.0,-0.18,2.07,4.86,X40N,ATH,FMCG
80,UNITDSPR,1644.00,-11.52,58.0,H-LC,7.27,231873.0,-2271.0,50896.0,0.51,-0.97,21.95,20.77,86.0,-0.04,1.65,5.88,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.08,42.0,H-LC,9.92,225609.0,-16315.0,79595.0,0.50,-6.74,35.28,26.15,8.0,-0.20,1.61,6.61,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.17,52.0,H-LC,3.07,160900.0,-25195.0,72679.0,-0.69,-13.54,45.17,25.51,15.0,-0.35,1.15,18.57,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
19,CIPLA,1795.00,-21.49,42.0,H-LC,5.13,209309.0,4809.0,36608.0,0.05,2.35,17.49,20.25,10.0,0.13,1.49,11.29,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-13.66,47.0,H-LC,4.45,216247.0,5881.0,22901.0,-0.26,2.80,10.59,13.68,37.0,0.26,1.54,19.78,XY25,NTT,REFINERIES
5,ASIANPAINT,3465.66,-13.98,44.0,H-LC,7.58,217272.0,-34496.0,87713.0,0.90,-13.70,40.37,21.13,27.0,-0.39,1.55,16.87,X40,ATH,PAINTS


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.04,41.0,H-LC,12.12,286362.0,-59594.0,134447.0,-0.52,-17.23,46.95,21.64,1.0,-0.44,2.04,1.45,X40,ATH,IT
40,INFY,2275.00,-18.52,47.0,H-LC,8.13,316944.0,4056.0,167632.0,-0.63,1.30,52.89,54.87,3.0,0.02,2.26,8.03,X40,BTT,IT
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196529.0,-3609.0,24507.0,-0.32,-1.80,12.47,10.44,4.0,-0.15,1.40,3.66,X40,NTT,FMCG
83,VBL,671.64,-19.29,34.0,H-LC,7.69,290896.0,-24946.0,137623.0,-0.56,-7.90,47.31,35.67,5.0,-0.18,2.07,4.86,X40N,ATH,FMCG
1,ABB,7934.00,-38.80,56.0,H-LC,8.59,255950.0,-5669.0,124878.0,0.63,-2.17,48.79,45.57,7.0,-0.05,1.82,8.91,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6983.33,61.0,L-SC,5.42,77700.0,-16110.0,92727.0,0.48,-17.17,119.34,81.67,269.0,-0.17,0.55,50.47,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,53.0,L-SC,42.80,86112.0,-27437.0,67486.0,0.76,-24.16,78.37,35.27,268.0,-0.41,0.61,110.76,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.86,41.0,H-SC,11.74,86710.0,-14253.0,108786.0,-0.39,-14.12,125.46,93.63,148.0,-0.13,0.62,29.15,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,49.0,H-SC,12.66,93885.0,-4095.0,25696.0,-0.46,-4.18,27.37,22.05,152.0,-0.16,0.67,36.47,XR,ATH,FINANCE
50,NATIONALUM,244.55,-44.49,61.0,H-MC,9.91,110897.0,10393.0,18719.0,0.19,10.34,16.88,28.96,79.0,0.56,0.79,47.82,MH,ATH,METALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1239.71,76.0,M-SC,9.96,182245.0,1102.0,44358.0,-1.44,0.61,24.34,25.10,235.0,0.02,1.30,46.05,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-14.58,73.0,H-SC,15.47,229469.0,-41292.0,371235.0,1.83,-15.25,161.78,121.86,133.0,-0.11,1.64,79.51,AR,ATH,IT
7,ATULAUTO,844.00,3764.86,56.0,M-SC,5.98,120454.0,-26914.0,78729.0,-1.96,-18.26,65.36,35.16,236.0,-0.34,0.86,23.03,XY24,NTT,AUTO
81,VAIBHAVGBL,521.00,59.23,59.0,H-SC,5.98,138056.0,-44719.0,155782.0,-0.96,-24.47,112.84,60.77,125.0,-0.29,0.98,23.65,XR,NTT,JEWELLERY
60,SAIL,228.00,49.96,72.0,M-MC,9.79,237309.0,12347.0,155319.0,0.37,5.49,65.45,74.53,192.0,0.08,1.69,39.51,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.19
1,25,43.71
2,50,74.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.1
LC    32.4
MC    23.5
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.37
X40      14.69
X40N     11.37
XR       11.30
XY25     10.02
AR        8.45
OX40N     7.83
X200      1.68
MH        1.64
X5K       1.42
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.49
H-LC    25.93
H-MC    20.52
M-SC    13.09
M-LC     5.43
M-MC     2.67
L-SC     1.52
L-LC     1.04
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.84,-5.69,40.58
IT,12.98,-15.55,78.55
FINANCE,10.08,-16.03,66.65
MISC,7.02,-16.60,81.42
BANKS,6.19,-15.05,75.58
PAINTS,5.65,-16.49,33.85
ELECTRICAL,5.48,-7.04,44.59
INSURANCE,3.72,-5.97,45.58
AC,3.54,2.59,13.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3121968.0,22
XR,1323556.0,14
AR,1146646.0,9
X40,786985.0,10
X40N,606917.0,9
OX40N,548739.0,10
XY25,433381.0,7
SR,249079.0,2
X5K,118253.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3658339.0,29
M-SC,1407689.0,17
H-MC,1226572.0,15
H-LC,1181921.0,15
M-LC,388029.0,4
M-MC,303428.0,2
L-SC,246308.0,3
L-MC,59411.0,1
L-LC,38111.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1224527.0      6
M-SC       XY24       803693.0      7
H-SC       AR         784116.0      5
           XR         773641.0      7
H-MC       XY24       563551.0      4
H-LC       X40        493894.0      5
M-MC       XY24       303428.0      2
H-SC       X40N       303035.0      4
           OX40N      258510.0      4
           SR         249079.0      2
H-LC       X40N       225127.0      3
H-MC       X40        220304.0      4
H-LC       AR         197557.0      2
H-MC       XY25       170341.0      2
M-SC       AR         164973.0      2
M-LC       XY24       161574.0      2
L-SC       XR         160213.0      2
M-SC       OX40N      134695.0      4
M-LC       X5K        118253.0      1
M-SC       XR         116626.0      2
           XY25       114915.0      1
H-LC       XY25       110014.0      3
M-LC       XR         108202.0      1
H-MC       XR         105463.0      1
H-LC       X200        90134.0      1
L-SC       OX40N       86095.0      1
H-MC       X40N        78755.0      2
M-SC       X40         72787.0      1
H-MC       OX40N       69439.0      1
H-SC       MH          65431.0      1
H-LC       XY24        65195.0      1
L-MC       XR          59411.0      1
L-LC       XY25        38111.0      1
H-MC       MH          18719.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
